# Conversational Database Agent - Testing Notebook

This notebook demonstrates the capabilities of the Conversational Database Agent for MongoDB.

## Setup and Installation

In [ ]:

# !pip install -r requirements.txt

In [ ]:
import os
import json
from datetime import datetime
from database_agent import ConversationalDatabaseAgent
from config import config

# Set environment variables (replace with your actual values)
os.environ["OPENAI_API_KEY"] = "openai-api-key"
os.environ["MONGODB_URI"] = "mongodb-connection-string"

## Mock Setup for Testing (if you don't have real connections)

In [ ]:

class MockMongoDBQueryExecutor:
    def __init__(self, connection_string: str, database_name: str = "sample_analytics"):
        self.data = {
            "accounts": [
                {"account_id": "A001", "limit": 10000, "products": ["checking", "savings"], "created_on": "2023-01-15"},
                {"account_id": "A002", "limit": 5000, "products": ["checking"], "created_on": "2023-02-20"},
                {"account_id": "A003", "limit": 15000, "products": ["checking", "credit"], "created_on": "2023-03-10"}
            ],
            "customers": [
                {"customer_id": "C001", "name": "John Doe", "email": "john@email.com", "active": True},
                {"customer_id": "C002", "name": "Jane Smith", "email": "jane@email.com", "active": True},
                {"customer_id": "C003", "name": "Bob Johnson", "email": "bob@email.com", "active": False}
            ],
            "transactions": [
                {"transaction_id": "T001", "account_id": "A001", "amount": 250.50, "date": "2024-01-15", "transaction_code": "DEP"},
                {"transaction_id": "T002", "account_id": "A002", "amount": -75.25, "date": "2024-01-16", "transaction_code": "WTH"},
                {"transaction_id": "T003", "account_id": "A001", "amount": 1200.00, "date": "2024-01-17", "transaction_code": "DEP"}
            ]
        }
    
    def execute_query(self, intent):
        from database_agent import QueryType
        
        if intent.query_type == QueryType.COUNT:
            collection = intent.collections[0] if intent.collections else "customers"
            count = len(self.data.get(collection, []))
            return [{"count": count, "collection": collection, "filters": intent.filters}]
        elif intent.query_type == QueryType.FILTER:
            collection = intent.collections[0] if intent.collections else "accounts"
            return self.data.get(collection, [])[:5]
        elif intent.query_type == QueryType.AGGREGATION:
            if intent.aggregation_type == "avg":
                return [{"average": 525.08}]
            elif intent.aggregation_type == "sum":
                return [{"total": 1375.25}]
        else:
            collection = intent.collections[0] if intent.collections else "accounts"
            return self.data.get(collection, [])[:3]

# Replace the real MongoDB executor with mock for testing
import database_agent
database_agent.MongoDBQueryExecutor = MockMongoDBQueryExecutor

## Initialize the Agent

In [ ]:
# Initialize the conversational database agent
agent = ConversationalDatabaseAgent(
    mongodb_uri="mongodb://demo:27017/",  # Replace with your MongoDB URI
    openai_api_key="demo_key"  # Replace with your OpenAI API key
)

print("Agent initialized successfully!")

## Test Basic Queries

In [ ]:
# Count query
session_id = f"notebook_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

query1 = "How many customers do we have?"
print(f"Query: {query1}")

try:
    response1 = agent.process_query(query1, session_id)
    print(f"Response: {response1['response']}")
    print(f"Query Type: {response1['query_type']}")
    print(f"Confidence: {response1['confidence']}")
    print(f"Insights: {response1['insights']}")
except Exception as e:
    print(f"Error: {e}")

In [ ]:
# Filter query
query2 = "Show me all accounts"
print(f"Query: {query2}")

try:
    response2 = agent.process_query(query2, session_id)
    print(f"Response: {response2['response']}")
    print(f"Query Type: {response2['query_type']}")
    print(f"Results Count: {response2['results_count']}")
except Exception as e:
    print(f"Error: {e}")

In [ ]:
# Aggregation query
query3 = "What's the average transaction amount?"
print(f"Query: {query3}")

try:
    response3 = agent.process_query(query3, session_id)
    print(f"Response: {response3['response']}")
    print(f"Query Type: {response3['query_type']}")
    print(f"Collections: {response3['collections_queried']}")
except Exception as e:
    print(f"Error: {e}")

## Test Conversation Memory

In [ ]:

queries = [
    "Tell me about the accounts collection",
    "How many of those accounts have a limit over 7000?",
    "What about customers?",
    "Show me the inactive ones"
]

print("Testing conversation memory:\n")

for i, query in enumerate(queries, 1):
    print(f"Query {i}: {query}")
    try:
        response = agent.process_query(query, session_id)
        print(f"Response: {response['response']}")
        if response['insights']:
            print(f"Insights: {', '.join(response['insights'])}")
        print("-" * 50)
    except Exception as e:
        print(f"Error: {e}")
        print("-" * 50)

## Session Insights Analysis

In [ ]:

try:
    insights = agent.get_session_insights(session_id)
    print("Session Insights:")
    print(json.dumps(insights, indent=2))
except Exception as e:
    print(f"Error getting insights: {e}")

## Test Schema Understanding

In [ ]:

schema_queries = [
    "What collections are available?",
    "Explain the transactions collection",
    "What fields are in the customers collection?"
]

print("Testing schema understanding:\n")

for query in schema_queries:
    print(f"Query: {query}")
    try:
        response = agent.process_query(query, session_id)
        print(f"Response: {response['response']}")
        print("-" * 50)
    except Exception as e:
        print(f"Error: {e}")
        print("-" * 50)

## Performance Testing

In [ ]:
import time

test_queries = [
    "Count customers",
    "Average transaction amount",
    "Show accounts",
    "Total transaction value",
    "Active customers count"
]

print("Performance Testing:\n")
response_times = []

for query in test_queries:
    start_time = time.time()
    try:
        response = agent.process_query(query, f"{session_id}_perf")
        end_time = time.time()
        response_time = end_time - start_time
        response_times.append(response_time)
        print(f"{query}: {response_time:.2f}s")
    except Exception as e:
        print(f"{query}: Error - {e}")

if response_times:
    avg_time = sum(response_times) / len(response_times)
    print(f"\nAverage response time: {avg_time:.2f}s")

## Error Handling Testing

In [ ]:

error_test_queries = [
    "What is the meaning of life?",  # Unrelated query
    "Show me unicorns",  # Non-existent data
    "Calculate the quantum flux",  # Meaningless query
    "",  # Empty query
    "asdf jkl; qwerty"  # Random text
]

print("Error Handling Testing:\n")

for query in error_test_queries:
    print(f"Query: '{query}'")
    try:
        response = agent.process_query(query, f"{session_id}_error")
        print(f"Response: {response['response']}")
        print(f"Query Type: {response['query_type']}")
        print(f"Confidence: {response['confidence']}")
    except Exception as e:
        print(f"Error handled: {e}")
    print("-" * 50)

## Summary

This notebook demonstrates the key features of the Conversational Database Agent:

1. **Natural Language Understanding**: Classifies user queries into different types
2. **Database Integration**: Executes appropriate MongoDB queries
3. **Conversation Memory**: Maintains context across multiple exchanges
4. **Insight Extraction**: Provides actionable insights from interactions
5. **Error Handling**: Gracefully handles ambiguous or invalid queries

The agent successfully maps natural language to database operations and provides intelligent responses with contextual awareness.